# RiskIQ PassiveTotal Python Library

#### *Using the Trackers dataset*

## Getting Started

This notebook leverages the RiskIQ Illuminate / PassiveTotal API through the `passivetotal` Python library. 

Documentation for the library, including how to install it and configure API keys, are available here:
https://passivetotal.readthedocs.io/en/latest/getting-started.html

You will need API credentials to authenticate with the API server that provide access to the datasets queried in this notebook. Ask your RiskIQ contact for details or visit https://info.riskiq.net/ to contact the support team.

### Optional Dependencies

This notebook uses the `pandas` Python library primarily to improve the visual output of data tables retrieved from the API. You will need to install that library in your Python (virtual) environment (`pip install pandas`) or change the code examples to return a Python dictionary instead of a dataframe. Simply change `.as_df` to `.as_dict`.

Note that some examples may use special features in `pandas` to filter or aggregate data, but these can also be implemented in pure Python.

### Product Context

[Trackers](https://info.riskiq.net/hc/en-us/articles/360057824494-PassiveTotal-Datasets-Trackers)
are unique codes or values found within web pages and often are used to track user interaction. These codes can be used to correlate a disparate group of websites to a central entity.


### Setup Notebook
*If this returns errors, ensure you have followed the Getting Started document linked above to install necessary dependencies and configure your API keys.*

In [1]:
from passivetotal import analyzer
analyzer.init()

### Table of Contents

* [Tracker History](#Tracker-History): Start with a hostname and get a history of trackers observed on that host.
* [Tracker Observations](#Tracker-Observations): Start with a tracker type and value to discover other sites where it has been observed.
* [Reference Trackers](#Reference-Trackers): Query a derived dataset starting with a host to find other hosts where a site's content has been copied and re-published.

---
## Tracker History

#### Hostname and IP Tracker History
RiskIQ gathers details on trackers during regular web crawls. The specific meaning of a tracker varies based on the "tracker type" assigned by RiskIQ analysts, but generally, one would expect a tracker value to uniquely identify a given site or organization. 

For example, a site admin may use a website monitoring product called New Relic to track the performance of their site. They will embedded Javascript code in their webpage that includes a uniquely-assigned identifier. RiskIQ crawlers will see and index that identifier as a tracker of type "NewRelicID" and associate the observation with the site where it was observed.

Here, we consider the trackers observed on www.irs.gov.

In [3]:
analyzer.set_date_range(days_back=30)

In [4]:
analyzer.Hostname('www.irs.gov').trackers.as_df

,query,firstseen,lastseen,trackertype,value,hostname
0,www.irs.gov,2017-08-30 05:34:35,2021-08-16 06:40:52,NewRelicId,b67fc6a152,www.irs.gov
1,www.irs.gov,2021-06-24 03:38:44,2021-08-15 11:24:47,TwitterShortlinkId,kcmyxuf42s,www.irs.gov
2,www.irs.gov,2021-06-24 03:38:44,2021-08-15 11:24:47,TwitterShortlinkId,ohcxvvmtsn,www.irs.gov
3,www.irs.gov,2021-07-23 04:09:09,2021-08-14 11:11:16,TwitterShortlinkId,hkypexoz6u,www.irs.gov
4,www.irs.gov,2021-07-23 04:09:09,2021-08-14 11:11:16,TwitterShortlinkId,ofswyypoai,www.irs.gov
5,www.irs.gov,2021-07-07 10:20:23,2021-07-22 12:43:57,TwitterShortlinkId,2cg1hmmb91,www.irs.gov


> NOTE: If you change the `days_back` value above and re-run the query, you won't get new a different set of results. This is due to caching in the `analyzer.Hostname` objects. You can restart the notebook kernel, or run `analyzer.Hostname('www.irs.gov').reset('trackers')` to clear the cache.

---
The `trackers` property of the `analyzer.Hostname` object returns an object of type `TrackerHistory` that behaves like a normal Python list, but also offers additional features through various properties. See the [reference docs](https://passivetotal.readthedocs.io/en/latest/analyzer.html?highlight=trackers#passivetotal.analyzer.trackers.TrackerHistory) for a complete list. 

In [8]:
for tracker in analyzer.Hostname('www.irs.gov').trackers:
    print(tracker)

[NewRelicId] "b67fc6a152" (2017-08-30 to 2021-08-16)
[TwitterShortlinkId] "kcmyxuf42s" (2021-06-24 to 2021-08-15)
[TwitterShortlinkId] "ohcxvvmtsn" (2021-06-24 to 2021-08-15)
[TwitterShortlinkId] "hkypexoz6u" (2021-07-23 to 2021-08-14)
[TwitterShortlinkId] "ofswyypoai" (2021-07-23 to 2021-08-14)
[TwitterShortlinkId] "2cg1hmmb91" (2021-07-07 to 2021-07-22)


Like most `analyzer` objects, each entry in a list of trackers can be treated like a string for easy display, but they also contain a set of properties and attributes for direct access to the data. These properties are explained in the [reference docs](https://passivetotal.readthedocs.io/en/latest/analyzer.html?highlight=trackers#passivetotal.analyzer.trackers.TrackerRecord) for `TrackerRecord` objects. 

In [9]:
analyzer.Hostname('www.irs.gov').trackers.filter(category='NewRelicId')[0].value

'b67fc6a152'

> The `trackers` property is also available for [IP addresses](https://passivetotal.readthedocs.io/en/latest/analyzer.html?highlight=trackers#ip-analysis), with similar functionalty, though in most cases we recommend starting with a fully-qualified domain name for best results.

---
## Tracker Observations

Trackers can be an effective way of discovering other internet sites controlled by legitimate entities, but it can also be used for threat investigations and phishing site detection.

When malicious actors copy website content with the intent to setup a phishing site, they often use automated tools that copy the entire HTML of the web page, including the Javascript and link parameters that setup trackers. In those cases, shared tracker values can be used to detect these copycat sites. 

The `analyzer` offers a top-level `Tracker` object you can use to search for all observations of a specific tracker type and value across hosts or IP addresses. 

In [13]:
analyzer.Tracker('NewRelicId','b67fc6a152').observations_by_hostname.as_df

,query,host,trackertype,firstseen,lastseen,searchtype
0,b67fc6a152,www.irsbgov.com,NewRelicId,2021-01-26 13:27:53,2021-01-26 18:12:09,hosts
1,b67fc6a152,www.irsqgov.com,NewRelicId,2021-01-23 15:31:21,2021-01-23 20:33:15,hosts
2,b67fc6a152,m2htc.com,NewRelicId,2021-06-14 08:03:57,2021-06-14 08:03:57,hosts
3,b67fc6a152,sa-www4-irs-gov.movementsinmotion.com,NewRelicId,2021-07-10 00:46:21,2021-07-10 07:45:48,hosts
4,b67fc6a152,rb3sports.com,NewRelicId,2021-06-22 03:14:12,2021-06-22 03:14:12,hosts
...,...,...,...,...,...,...
135,b67fc6a152,dezurahomes.com,NewRelicId,2021-06-23 22:59:43,2021-06-23 22:59:43,hosts
136,b67fc6a152,1rs-gov.org,NewRelicId,2021-06-07 13:28:32,2021-06-11 18:31:34,hosts
137,b67fc6a152,xaayvi.icu,NewRelicId,2020-06-23 17:58:39,2020-06-23 17:58:39,hosts
138,b67fc6a152,arteservices.com,NewRelicId,2021-06-22 22:13:57,2021-06-22 22:13:57,hosts


The `analyzer.Tracker` object provides two properties to aid discovery of related sites: `observations_by_hostname` and `observations_by_ip`. Both return a list of observations as a `TrackerSearchResults` object that offers many of the same capabilities as a `TrackerHistory` object. 

You can instantiate a `analyzer.Tracker` object directly as shown above, or obtain an instance from the `tracker` property of a record returned in the `TrackerHistory` of a hostname or IP address.

In [18]:
analyzer.Hostname('www.irs.gov').trackers.filter(category='NewRelicId')[0].tracker

<Tracker NewRelicId:b67fc6a152>

In [20]:
(
    analyzer.Hostname('www.irs.gov')
    .trackers
    .filter(category='NewRelicId')[0]
    .tracker
    .observations_by_hostname
    .totalrecords
)

140

> This syntax can be a bit strange when you first encounter it. Python style guides generally discourage long lines of code, but when they are unavoidable or justified, the syntax permits enclosing blocks in parentheses.  

These observations show other sites where RiskIQ has observed the same value for the NewRelicId tracker that the IRS has configured on their site. If these observations are subdomains of the 'irs.gov' domain they are likely benign, but if not, they are suspicious and worth further research.

We can leverage features of the `analyzer` module and these specific tracker objects to focus on those suspicious sites.

In [2]:
whitelist = ['irs.gov','translate.goog','t.co']
suspicious_trackers = (
    analyzer.Tracker('NewRelicId','b67fc6a152')
    .observations_by_hostname
    .exclude_domains_in(whitelist)
)
suspicious_trackers.as_df

,query,host,trackertype,firstseen,lastseen,searchtype
0,b67fc6a152,www.irsbgov.com,NewRelicId,2021-01-26 13:27:53,2021-01-26 18:12:09,hosts
1,b67fc6a152,www.irsqgov.com,NewRelicId,2021-01-23 15:31:21,2021-01-23 20:33:15,hosts
2,b67fc6a152,m2htc.com,NewRelicId,2021-06-14 08:03:57,2021-06-14 08:03:57,hosts
3,b67fc6a152,sa-www4-irs-gov.movementsinmotion.com,NewRelicId,2021-07-10 00:46:21,2021-07-10 07:45:48,hosts
4,b67fc6a152,rb3sports.com,NewRelicId,2021-06-22 03:14:12,2021-06-22 03:14:12,hosts
...,...,...,...,...,...,...
124,b67fc6a152,dezurahomes.com,NewRelicId,2021-06-23 22:59:43,2021-06-23 22:59:43,hosts
125,b67fc6a152,1rs-gov.org,NewRelicId,2021-06-07 13:28:32,2021-06-11 18:31:34,hosts
126,b67fc6a152,xaayvi.icu,NewRelicId,2020-06-23 17:58:39,2020-06-23 17:58:39,hosts
127,b67fc6a152,arteservices.com,NewRelicId,2021-06-22 22:13:57,2021-06-22 22:13:57,hosts


> By default, `pandas` will only show a subset of rows in notebooks. To display more, first `import pandas as pd` and then run `pd.options.display_max_rows=500`.

> `host.registered_domain` works because the `host` attribute of tracker record returns an object of type `analyzer.Hostname`, and those objects offer several properties provided by the `tldextract` Python library, including `tld` and `registered_domain`.

As a further validation, we could examine the age of these domains and the registrant owner using whois data available in the `whois` property of the hostnames.

In [37]:
suspicious_tracker_analysis = []
for tracker in suspicious_trackers.sorted_by('lastseen', True)[0:5]:
    analysis = { 
        'host': str(tracker.host),
        'whois_age': tracker.host.whois.age,
        'whois_org': tracker.host.whois.registrant_org.value
    }
    suspicious_tracker_analysis.append(analysis)
suspicious_tracker_analysis

[{'host': 'payment.irs.benefit.marypoesia.com',
  'whois_age': 447,
  'whois_org': 'Whois Privacy Protection Foundation'},
 {'host': 'www8irs-gov.seoorg.ro', 'whois_age': 488, 'whois_org': None},
 {'host': 'irservice.online',
  'whois_age': 14,
  'whois_org': 'Privacy Protect, LLC (PrivacyProtect.org)'},
 {'host': 'swwvv4-irs-gov.seoorg.ro', 'whois_age': 488, 'whois_org': None},
 {'host': 'www.interrservice.com', 'whois_age': 19, 'whois_org': None}]

> The `tracker.host` object will return an `analyzer.Hostname` object. Cast it as a string to get just the text value.

> The second parameter of `sorted_by('lastseen', True)` activates a reverse sort, and together with the slice notion `[0:5]` gives us the top 5 `TrackerSearchRecord` objects. 

## Reference Trackers

RiskIQ researchers have identified several instances where the value of a tracker provides an indication of where an Internet asset was originally hosted or where an Internet asset’s response body was originally copied from. We have merged these identifiers into our tracker dataset under one of several categories (or types).

In the `analyzer` these trackers are available in the `trackers_reference` property of `analyzer.Hostname` and `analyzer.IPAddress` objects.

Among other use cases, this enables you to find websites hosting files that were originally downloaded from a given site, often with malicious intent.

In [2]:
analyzer.Hostname('www.irs.gov').tracker_references.as_df

,query,host,trackertype,firstseen,lastseen,searchtype
0,www.irs.gov,142.250.80.65,DocumentBaseHost,2021-07-16 10:22:55,2021-07-27 10:09:18,addresses
1,www.irs.gov,172.217.1.129,DocumentBaseHost,2021-05-03 20:35:20,2021-05-03 20:35:20,addresses
2,www.irs.gov,173.194.211.132,DocumentBaseHost,2021-02-22 14:26:13,2021-06-21 22:36:51,addresses
3,www.irs.gov,172.217.1.97,DocumentBaseHost,2021-05-13 00:00:59,2021-05-13 00:00:59,addresses
4,www.irs.gov,172.217.165.129,DocumentBaseHost,2021-02-22 11:07:16,2021-02-22 11:07:16,addresses
...,...,...,...,...,...,...
117,www.irs.gov,tax50.net,MarkOfTheWebSourceHost,2021-04-06 00:47:06,2021-04-06 00:47:07,hosts
118,www.irs.gov,severvice0utkook.gq,MarkOfTheWebSourceHost,2019-07-21 03:55:01,2019-07-21 03:55:01,hosts
119,www.irs.gov,www.tax50.net,MarkOfTheWebSourceHost,2019-04-28 17:38:09,2021-04-06 00:46:18,hosts
120,www.irs.gov,applygvttaxrdcn.multiversehobbies.com,MarkOfTheWebSourceHost,2018-11-21 12:31:32,2018-11-21 23:45:55,hosts


> This property performs multiple API queries to search both IPs and hosts for several types of trackers. If you need to conserve API queries, instantiate an `analyzer.Tracker` object and use the `observations_by_hostname` or `observations_by_ip` properties directly.

Combining features from `pandas` and the `analyzer` module, we can create a custom dataframe with the RiskIQ Illuminate Reputation Score for each domain. 

In [5]:
whitelist = ['irs.gov','translate.goog','t.co']
tracker_df = (
    analyzer.Hostname('www.irs.gov')
    .tracker_references
    .filter(searchtype='hosts')
    .exclude_domains_in(whitelist)
    .as_df
)
tracker_df['reputation_score'] = tracker_df.apply(
    lambda row: analyzer.Hostname(str(row['host'])).reputation.score, 
    axis=1
)
del(tracker_df['query'])
del(tracker_df['searchtype'])
tracker_df.nlargest(10,'reputation_score')

,host,trackertype,firstseen,lastseen,reputation_score
10,irs-tax.online,HTTrackSourceHost,2021-06-28 14:02:33,2021-06-28 15:03:43,100
2,irservice.online,HTTrackSourceHost,2021-08-03 18:12:45,2021-08-03 19:27:54,74
13,severvice0utkook.cf,MarkOfTheWebSourceHost,2019-07-21 03:54:56,2019-07-21 03:54:56,74
18,recentchecking.cf,MarkOfTheWebSourceHost,2019-07-08 13:03:57,2019-07-08 16:25:00,71
7,hanhphuctretho-org.micnguyen.com,HTTrackSourceHost,2020-06-14 12:31:48,2020-06-14 12:31:48,69
15,severvice0utkook.gq,MarkOfTheWebSourceHost,2019-07-21 03:55:01,2019-07-21 03:55:01,69
12,service03.ml,MarkOfTheWebSourceHost,2019-07-18 21:49:00,2019-07-22 16:32:38,68
6,3aconsultores.mx,HTTrackSourceHost,2020-02-25 19:16:07,2020-02-25 20:45:49,54
0,hanhphuctretho.org,HTTrackSourceHost,2020-06-13 20:16:20,2020-06-14 23:12:42,53
1,interrservice.com,HTTrackSourceHost,2021-07-30 01:50:58,2021-07-30 01:50:59,0


The `reputation` property of Hostnames and IPAddress objects includes a `rules` property that offers insight into how the score was calculated. We can access the property directly or display it using `pandas`.

In [67]:
analyzer.Hostname('severvice0utkook[.]cf').reputation.to_dataframe(explode_rules=True)

,query,score,classification,name,description,severity,link
0,severvice0utkook.cf,74,SUSPICIOUS,TLD,Domains in this TLD are more likely to be mali...,4,None
1,severvice0utkook.cf,74,SUSPICIOUS,ASN,Infrastructure hosted by this ASN are more lik...,3,None


> The `as_df` property is a shortcut to the `to_dataframe()` method available on nearly all `analyzer` objects. In some cases, `to_dataframe()` offers unique behavior specific to the object it is acting on. Here, we use a `pandas.DataFrame.explode()` method to unpack a list of rules and present them as rows, hence the `explode_rules` parameter.